clustering final --> debug for api

In [21]:
import sys
sys.path.append('../../../')

In [22]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

from modules import machine_learning_utils as mlu

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

# 1.0 Data retrieval

In [23]:
df = pd.read_pickle("../../../data_lake/output_prod/train.pkl")

# 2.0 Features Engineering

In [24]:
df = mlu.features_eng(df,'clustering')

In [25]:
cols = ['card1', 'card2', 'card3', 'card5', 'M4', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25',
       'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_32', 'id_33', 'id_34',
       'id_35', 'id_36', 'id_37', 'id_38','num_transaction_per_time','addr1','P_emaildomain','R_emaildomain']

In [26]:
df = df.drop(columns=cols)

In [27]:
df = df[df['isFraud']==0]
df = df.drop(columns='isFraud')

In [28]:
df = df.drop(columns=['addr2','multi_transaction_per_time'])

In [29]:
df.head(1)

,TransactionAmt,ProductCD,card4,card6,DeviceType,num_accounts_related_to_user,num_days_previous_transaction,customer_id,browser_enc,device_info_v4
139309,28.588,C,visa,debit,mobile,1.0,0.0,116,chrome,other


In [30]:
with open("../../../data_lake/output_prod/clustering_imputation_cat.pkl","rb") as imc:
    imputation_cat = pickle.load(imc)

with open("../../../data_lake/output_prod/clustering_imputation_num.pkl","rb") as imn:
    imputation_num = pickle.load(imn)

with open("../../../data_lake/output_prod/clustering_scaler_num.pkl","rb") as sn:
    scaler_num = pickle.load(sn)
    
with open("../../../data_lake/output_prod/clustering_model_v1.pkl","rb") as mod:
    model = pickle.load(mod)

# Testing

In [31]:
test = df.head(1)

In [32]:
test

,TransactionAmt,ProductCD,card4,card6,DeviceType,num_accounts_related_to_user,num_days_previous_transaction,customer_id,browser_enc,device_info_v4
139309,28.588,C,visa,debit,mobile,1.0,0.0,116,chrome,other


In [33]:
test.dtypes

TransactionAmt                   float64
ProductCD                         object
card4                             object
card6                             object
DeviceType                        object
num_accounts_related_to_user     float64
num_days_previous_transaction    float64
customer_id                        int64
browser_enc                       object
device_info_v4                    object
dtype: object

In [34]:
test = mlu.clustering_preparation(test,'prediction',imputation_num,scaler_num,imputation_cat)

../../../modules/machine_learning_utils.py:426: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

../../../modules/machine_learning_utils.py:427: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

../../../modules/machine_learning_utils.py:426: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [35]:
test.dtypes

TransactionAmt                   float64
ProductCD                         object
card4                             object
card6                             object
DeviceType                        object
num_accounts_related_to_user     float64
num_days_previous_transaction    float64
customer_id                        int64
browser_enc                       object
device_info_v4                    object
dtype: object

In [36]:
test

,TransactionAmt,ProductCD,card4,card6,DeviceType,num_accounts_related_to_user,num_days_previous_transaction,customer_id,browser_enc,device_info_v4
139309,0.01674,C,visa,debit,mobile,0.000586,0.0,116,chrome,other


In [37]:
test = mlu.clustering_encoding(test,'prediction')

../../../modules/machine_learning_utils.py:439: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

../../../modules/machine_learning_utils.py:440: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

../../../modules/machine_learning_utils.py:441: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [38]:
test.dtypes

TransactionAmt                   float64
num_accounts_related_to_user     float64
num_days_previous_transaction    float64
customer_id                        int64
product_enc                        int64
card4_enc                          int64
card6_enc                          int64
DeviceType_enc                     int64
browser_enc2                       int64
device_info_v4_enc                 int64
dtype: object

In [39]:
test

,TransactionAmt,num_accounts_related_to_user,num_days_previous_transaction,customer_id,product_enc,card4_enc,card6_enc,DeviceType_enc,browser_enc2,device_info_v4_enc
139309,0.01674,0.000586,0.0,116,0,0,0,0,0,17


In [57]:
model.predict(X=test[['TransactionAmt', 'num_accounts_related_to_user', 'num_days_previous_transaction',
                                                    'product_enc', 'card4_enc', 'card6_enc', 'DeviceType_enc','browser_enc2',
                                                    'device_info_v4_enc']],categorical=[3,4,5,6,7,8])[0]

2

In [41]:
test

,TransactionAmt,num_accounts_related_to_user,num_days_previous_transaction,customer_id,product_enc,card4_enc,card6_enc,DeviceType_enc,browser_enc2,device_info_v4_enc,cluster_labels_pred
139309,0.01674,0.000586,0.0,116,0,0,0,0,0,17,2


In [42]:
test.dtypes

TransactionAmt                   float64
num_accounts_related_to_user     float64
num_days_previous_transaction    float64
customer_id                        int64
product_enc                        int64
card4_enc                          int64
card6_enc                          int64
DeviceType_enc                     int64
browser_enc2                       int64
device_info_v4_enc                 int64
cluster_labels_pred               uint16
dtype: object

In [61]:
t = pd.DataFrame(columns=test.columns)

In [62]:
t

,TransactionAmt,num_accounts_related_to_user,num_days_previous_transaction,customer_id,product_enc,card4_enc,card6_enc,DeviceType_enc,browser_enc2,device_info_v4_enc,cluster_labels_pred


In [60]:
test.columns

Index(['TransactionAmt', 'num_accounts_related_to_user',
       'num_days_previous_transaction', 'customer_id', 'product_enc',
       'card4_enc', 'card6_enc', 'DeviceType_enc', 'browser_enc2',
       'device_info_v4_enc', 'cluster_labels_pred'],
      dtype='object')

In [ ]:
for i in test:
    t[i]